[View in Colaboratory](https://colab.research.google.com/github/RaulMedeiros/Deep_Transfer_Learning_Framework/blob/master/Classification_Framework.ipynb)

In [0]:
src_folder_id = '1dQHvtUYX_HFXCH6lxSaYfN36_Vztncpv'
out_folder_id = '1JqEVlTzcO6m2Fwvd-yyudntbEuCtVENY'

#Especifique as topologias a serem utilizadas.
model_name_list = ['MobileNet','ResNet50','Xception','VGG16','VGG19',
                    'InceptionV3','InceptionResNetV2','NASNetMobile','NASNetLarge',
                    'DenseNet121','DenseNet169','DenseNet201']

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
!rm -r data
!pip3 install -q pandas
!mkdir data

import pandas as pd

def load_dataset(folder_id):
  """Download all files into a folder"""   
  info = pd.DataFrame(drive.ListFile({'q': "title contains '.npy' and '" +folder_id +"' in parents and trashed=false" }).GetList())    
 
  for idx, title in zip(info['id'].values,info['title'].values):
    downloaded = drive.CreateFile({'id': idx })
    downloaded.GetContentFile('./data/'+title[5:])  ## !!!!!!!!!!!!!!!!!!!./out/
   
  !ls data
  return True

#Load and Unconpress data
load_dataset(src_folder_id)

DenseNet121.npy  InceptionResNetV2.npy	NASNetLarge.npy   VGG16.npy
DenseNet169.npy  InceptionV3.npy	NASNetMobile.npy  VGG19.npy
DenseNet201.npy  MobileNet.npy		ResNet50.npy	  Xception.npy


True

In [0]:
# https://keras.io/
!pip install -q keras
import keras

from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn import metrics

from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm


import itertools
from keras.utils import np_utils
from scipy.stats import randint as sp_randint


import numpy as np
import glob
import random
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')
#     print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def calc_scores(clf, X, y,clf_name,class_names, cv, plot_cnf_matrix, figsize, normalize):
    
#     ## Compute Evaluation Metrics
#     scores = cross_val_score(clf, X, y, cv=cv)
#     print(clf_name,np.mean(scores))
        
    scoring = ['accuracy', 'precision','recall','f1','roc_auc']
    scores = cross_validate(clf, X, y, cv=cv ,scoring=scoring)
    print('\n',clf_name,'\nAccuracy',np.array(np.mean(scores['test_accuracy'])),
                        '\nPrecision',np.array(np.mean(scores['test_precision'])),                     
                        '\nRecall',np.array(np.mean(scores['test_recall'])),
                        '\nF1',np.array(np.mean(scores['test_f1'])),
                        '\nROC_AUC',np.array(np.mean(scores['test_roc_auc'])))
    
    means = [np.array(np.mean(scores['test_accuracy'])),
#              np.array(np.mean(scores['test_precision'])),
#              np.array(np.mean(scores['test_recall'])),
#              np.array(np.mean(scores['test_f1'])),
             np.array(np.mean(scores['test_roc_auc']))]
    
    print('means:',np.mean(np.array(means)))
    
    
    ## Compute Confusion matrix
    y_pred = cross_val_predict(clf,X,y,cv=cv)
    cnf_matrix = confusion_matrix(y,y_pred)
    
    # Plot Confusion matrix
    if (plot_cnf_matrix):
        plt.figure(figsize=figsize)
        if (normalize):
            title = ' Normalized confusion matrix'
        else:
            title = ' non-Normalized confusion matrix'
        class_names = ['0']
        plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=normalize,title=clf_name+title)
        plt.show()
    
def load_Ds(folder_Path):
    ds = np.load(folder_Path)
    X = np.array(ds[:,:-1])
    y = np.array(list(map(int,ds[:,-1])))
    return X,y

#Normalized Data
def normalizeX(X):        
    X = np.array(X - np.mean(X))
    X = np.array(((X-np.min(X))/(np.max(X)-np.min(X))))     
    return X

def shuffle(x_data, y_data):
    c = list(zip(x_data, y_data))
    random.shuffle(c)
    X,y = zip(*c)
    X = np.array(X)
    y = np.array(y)
    return X,y

def simplifly(X,y):   
    idx_list = [idx for idx, k in enumerate(y) if k!=2]    
    X = np.array(X[idx_list])
    y = np.array(y[idx_list])
    
    # Binary problem
    y = np.array([0 if k<2 else k for k in y])
#     y = np.array([1 if k==2 else k for k in y])        
    y = np.array([1 if k>2 else k for k in y])
    
    # Multiclas problem
#     y = np.array([2 if k==3 else k for k in y])
#     y = np.array([3 if k==4 else k for k in y])
    return X, y

In [0]:
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import f1_score
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import make_scorer

# scoring = {'accuracy': make_scorer(accuracy_score),
#            'f1_score': make_scorer(f1_score),
#            'confusion_matrix': make_scorer(confusion_matrix)}

#  scores = cross_validate(clf, X, y, cv=cv ,scoring=scoring)    
#  print('\n',clf_name,'\t',np.round(np.array(np.mean(scores['test_accuracy'])), 4))
# #        '\t',np.round(np.array(np.mean(scores['test_f1_score'])), 4),
# #        '\t',np.round(np.array(np.mean(scores['test_confusion_matrix'])), 4))

In [14]:
% time

from IPython.display import clear_output

from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.metrics import *

def tp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 0]
def tn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 0]
def fp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 0]
def fn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 1]


class_weight = {0: 1 ,1: 1}

#Especifique se a matriz de confusão deve ser plotada
plot_cnf_matrix= False
#Especifique se a matriz de confusão plotada deve ser normalizada
normalize =True 
#Especifique o tamanho matriz de confução deve ser plotada
figsize = (5,5) # <<normal # figsize = (10,10) <<<Grande
LOG = False
class_names = ['0','1''2','3','4','5','6','7','8','9','10','11','12','13','14']
pre = 4

# KFOLD 
cv = 10

clf_list = np.array([ 
            MultinomialNB(),        
            MLPClassifier(max_iter=1000,solver='adam',learning_rate_init=5e-04),      
            KNeighborsClassifier(),   
            RandomForestClassifier(class_weight=class_weight),
            svm.SVC(kernel='linear',class_weight=class_weight,probability=True,max_iter=3000,tol=1e-3),
            svm.SVC(kernel='poly',class_weight=class_weight,probability=True,max_iter=3000,tol=1e-3),
            svm.SVC(kernel='rbf',class_weight=class_weight,probability=True,max_iter=3000,tol=1e-3),
           ])


# specify parameters and distributions to sample from
param_dist_list = np.array([ 
                    #Bayes
                    None, 
                    {"hidden_layer_sizes": list(np.arange(2,1001))},
                    # KNN
                    {"n_neighbors": [1,3,5,7,9,11]}, 
                    # Random Forest                   
                    {  "n_estimators": [3000],
                       "max_depth": [6, None],
                       "max_features": sp_randint(1, 11),
                       "min_samples_split": sp_randint(2, 11),
                       "min_samples_leaf": sp_randint(1, 11),
                       "bootstrap": [True, False],
                       "criterion": ["gini", "entropy"]}
                    ,
                    # SVM Linear
                    {'kernel': ['linear'], 'C': [2**i for i in range(-5,15)]},
                    # SVM Polynomial    
                    {'kernel': ['poly'], 'degree': [3, 5, 7 ,9], 'C': [2**i for i in range(-5,15)]},                    
                    # SVM RBF    
                    {'kernel': ['rbf'], 'gamma': [2**i for i in range(-15,3)], 'C': [2**i for i in range(-5,15)]},
                    ])

clf_name_list = np.array([
                 'Bayes',              ## 0
                 'MLP',                ## 1
                 'Nearest_Neighbors',  ## 2
                 'Random_Forest',      ## 3
                 'SVM_Linear',         ## 4
                 'SVM_Polynomial',     ## 5
                 'SVM_RBF',            ## 6
                ])
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision_macro': 'precision_macro',
           'recall_macro':'recall_macro',
           'f1_macro':  'f1_macro',
           'matthews_corrcoef':make_scorer(matthews_corrcoef)
          }


# Config classifiers
# idx_clf = np.array(range(7))
idx_clf = np.array([0,1,4,6])
clf_name_list = clf_name_list[idx_clf]
clf_list = clf_list[idx_clf]
param_dist_list = param_dist_list[idx_clf]

# number of interaction of 'random Search'
n_iter_search_list = np.array([0,20,5,15,20,20,20])
n_iter_search_list = n_iter_search_list[idx_clf]


df_kfolds_scores = pd.DataFrame()
df_mean_scores = pd.DataFrame()

for model_name in model_name_list:
  list_df = []
  print('processing: '+model_name+' deep model')

  X, y  = load_Ds('./data/'+model_name+'.npy')
  X = normalizeX(X)           
  X, y  = shuffle(X, y)

  if(LOG):
      print("\nLoaded DS has:",X.shape,"intances",y.shape,"labels")
      print("Simplifly DS has:",X.shape,"intances",y.shape,"labels")
      num_classes = len(np.unique(y))
      classes = [np.sum([1 for x in y if x==k]) for k in range(num_classes)]
      print("Number of Classes:",num_classes,"with",classes,"elements respectively.")


  for idx, (clf ,clf_name,param_dist,n_iter_search) in enumerate(zip(clf_list,clf_name_list,param_dist_list,n_iter_search_list)): 
    if(param_dist != None): 
        random_search = RandomizedSearchCV(clf, param_distributions=param_dist,n_iter=n_iter_search,n_jobs=-1,cv=2)
        random_search.fit(X, y)
        clf = random_search.best_estimator_
#     else:
#         clf.fit(X, y)

    # Compute metrics with the best classifier configuration
    scores = cross_validate(clf, X, y, cv=cv ,scoring=scoring)   

    # Format log Dataframe 
    df = pd.DataFrame(scores) 
    ds_mean = df.mean()
    ds_std = df.std() 
    df.insert(0, 'model', [model_name for i in range(cv)])
    df.insert(1, 'classifier', [clf_name for i in range(cv)])  
    df_kfolds_scores = df_kfolds_scores.append(df,ignore_index=True)
    df.loc['mean'] = ds_mean    
    df.loc['std'] = ds_std   
    df = df.drop(columns=['model', 'classifier'])
    df.insert(0, 'model', [model_name for i in range(cv+2)])
    df.insert(1, 'classifier', [clf_name for i in range(cv+2)]) 
    list_df.append(df)

  
  # Display metrics summary
  clear_output()    
  df_total = pd.concat(list_df)
  mean_std = df_total.drop(range(cv))
  display(mean_std)

  # Authenticate and create the PyDrive client.
  # This only needs to be done once per notebook.
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  
  #Save log Dataframes into csv files
  df_total.to_csv('./result_total_'+str(model_name)+'.csv', sep=',', mode='a')
  mean_std.to_csv('./result_mean_std_'+str(model_name)+'.csv', sep=',', mode='a')
  !ls

  #Upload npy file to folder (by id)
  file = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": out_folder_id}]})
  file.SetContentFile('./result_total_'+str(model_name)+'.csv')
  file.Upload() 

  #Upload npy file to folder (by id)
  file = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": out_folder_id}]})
  file.SetContentFile('./result_mean_std_'+str(model_name)+'.csv')
  file.Upload() 

,model,classifier,fit_time,score_time,test_accuracy,test_f1_macro,test_matthews_corrcoef,test_precision_macro,test_recall_macro,train_accuracy,train_f1_macro,train_matthews_corrcoef,train_precision_macro,train_recall_macro
mean,DenseNet201,Bayes,0.008232,0.006070,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
std,DenseNet201,Bayes,0.000707,0.000293,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,DenseNet201,MLP,32.176362,0.070135,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
std,DenseNet201,MLP,0.898158,0.002293,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,DenseNet201,SVM_Linear,2.737323,0.268332,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
std,DenseNet201,SVM_Linear,0.017306,0.003767,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,DenseNet201,SVM_RBF,5.045208,0.337050,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
std,DenseNet201,SVM_RBF,0.024629,0.001427,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


data				       result_mean_std_Xception.csv
datalab				       result_total_DenseNet121.csv
out				       result_total_DenseNet169.csv
result_mean_std_DenseNet121.csv        result_total_DenseNet201.csv
result_mean_std_DenseNet169.csv        result_total_InceptionResNetV2.csv
result_mean_std_DenseNet201.csv        result_total_InceptionV3.csv
result_mean_std_InceptionResNetV2.csv  result_total_MobileNet.csv
result_mean_std_InceptionV3.csv        result_total_NASNetLarge.csv
result_mean_std_MobileNet.csv	       result_total_NASNetMobile.csv
result_mean_std_NASNetLarge.csv        result_total_ResNet50.csv
result_mean_std_NASNetMobile.csv       result_total_VGG16.csv
result_mean_std_ResNet50.csv	       result_total_VGG19.csv
result_mean_std_VGG16.csv	       result_total_Xception.csv
result_mean_std_VGG19.csv


In [0]:
# ['accuracy', 
#  'adjusted_mutual_info_score', 
#  'adjusted_rand_score',
#  'average_precision',
#  'completeness_score',
#  'explained_variance', 
#  'f1',
#  'f1_macro', 
#  'f1_micro', 
#  'f1_samples',
#  'f1_weighted',
#  'fowlkes_mallows_score',
#  'homogeneity_score',
#  'mutual_info_score',
#  'neg_log_loss',
#  'neg_mean_absolute_error',
#  'neg_mean_squared_error',
#  'neg_mean_squared_log_error',
#  'neg_median_absolute_error',
#  'normalized_mutual_info_score',
#  'precision', 
#  'precision_macro', 
#  'precision_micro',
#  'precision_samples', 
#  'precision_weighted', 
#  'r2', 
#  'recall', 
#  'recall_macro',
#  'recall_micro', 
#  'recall_samples', 
#  'recall_weighted', 
#  'roc_auc', 
#  'v_measure_score']

In [9]:
print('Done')

data  datalab  result_kfold_Xception.csv  result_mean_std_Xception.csv
